In [5]:
## 🚀 Setup and Helper Functions

import json
from datetime import datetime, timedelta
import glob
import os

# --- Helper function for timestamp manipulation ---

def get_new_timestamps(timestamps):
    """
    Rounds a list of timestamps to the nearest 500 milliseconds (0, 500)
    and ensures the resulting list is strictly increasing, using a minimum
    100ms step to resolve collisions.

    Args:
        timestamps (list): A list of original timestamp strings.

    Returns:
        list: A list of new, unique, rounded timestamp strings,
              in the same chronological order as the input.
    """
    if not timestamps:
        return []

    # Convert original timestamps to datetime objects for sorting and manipulation
    datetime_objects = sorted([
        datetime.fromisoformat(ts.replace('Z', '+00:00')) for ts in timestamps
    ])

    new_datetime_objects = []
    previous_timestamp = None

    for timestamp in datetime_objects:

        # 1. Calculate the closest half-second mark (Target Time)
        milliseconds = timestamp.microsecond // 1000
        remainder = milliseconds % 500

        if remainder < 250:
            milliseconds -= remainder
        else:
            milliseconds += (500 - remainder)

        # Handle overflow: e.g., 999ms rounds up to 1000ms (which is 1 second)
        if milliseconds == 1000:
            rounded_timestamp = timestamp + timedelta(seconds=1)
            rounded_timestamp = rounded_timestamp.replace(microsecond=0)
        else:
            rounded_timestamp = timestamp.replace(microsecond=milliseconds * 1000)

        # 2. Ensure strictly increasing uniqueness with 100ms increment
        # If the target rounded time is less than or equal to the previous,
        # shift it forward by 100ms from the previously assigned time.
        if previous_timestamp and rounded_timestamp <= previous_timestamp:
            # Collision detected. Shift forward by 100ms.
            rounded_timestamp = previous_timestamp + timedelta(milliseconds=100)

        previous_timestamp = rounded_timestamp
        new_datetime_objects.append(rounded_timestamp)

    # 3. Convert back to the required string format
    new_timestamps_str = [
        ts.isoformat().replace('+00:00', 'Z') for ts in new_datetime_objects
    ]

    return new_timestamps_str

def update_timestamps(data):
    """
    Updates timestamps in 'gameMoves' and 'doSomethingTimestamps' ensuring
    no two final timestamps are the same across the entire file.

    Args:
        data (dict): The loaded JSON data from a game file.

    Returns:
        dict: The data dictionary with updated timestamps.
    """

    # 1. Collect all timestamps and corresponding values
    original_game_moves = data['gameMoves']
    original_do_something_timestamps = data['doSomethingTimestamps']

    # Combine all original timestamps for global chronological processing
    all_original_timestamps = sorted(
        list(original_game_moves.keys()) + list(original_do_something_timestamps.keys())
    )

    # 2. Generate the globally unique and sorted new timestamps
    all_new_timestamps = get_new_timestamps(all_original_timestamps)

    # Create an ordered mapping: Original TS -> New TS
    ts_mapping = dict(zip(all_original_timestamps, all_new_timestamps))

    # 3. Rebuild the two dictionaries using the mapping, guaranteeing no loss
    new_game_moves = {}
    for original_ts, move_value in original_game_moves.items():
        new_ts = ts_mapping[original_ts]
        new_game_moves[new_ts] = move_value

    new_do_something_timestamps = {}
    for original_ts, action_value in original_do_something_timestamps.items():
        new_ts = ts_mapping[original_ts]
        new_do_something_timestamps[new_ts] = action_value

    # 4. Update the data dictionary
    data['gameMoves'] = new_game_moves
    data['doSomethingTimestamps'] = new_do_something_timestamps

    return data

# --- Main processing function ---

def process_files(file_list, output_prefix='_updated'):
    """
    Processes a list of JSON files to update timestamps and saves the
    result to a new file with the specified prefix.
    """
    print(f"--- Starting file processing for {len(file_list)} files ---")

    for input_filepath in file_list:
        try:
            # 1. Determine the output filename
            directory = os.path.dirname(input_filepath)
            filename_with_ext = os.path.basename(input_filepath)
            name, ext = os.path.splitext(filename_with_ext)

            # Construct the new filename: insert '_updated' before the final number
            last_underscore_index = name.rfind('_')
            if last_underscore_index != -1 and name[last_underscore_index+1:].isdigit():
                base_name = name[:last_underscore_index]
                suffix = name[last_underscore_index:]
                new_name = f"{base_name}{output_prefix}{suffix}"
            else:
                new_name = f"{name}{output_prefix}"

            output_filepath = os.path.join(directory, f"{new_name}{ext}")

            print(f"\nProcessing: {input_filepath}")

            # 2. Load the data
            with open(input_filepath, 'r') as f:
                data = json.load(f)

            # Check original lengths before update
            original_gm_len = len(data['gameMoves'])
            original_dst_len = len(data['doSomethingTimestamps'])
            total_original = original_gm_len + original_dst_len

            # 3. Update the timestamps
            updated_data = update_timestamps(data)

            # Check final lengths after update
            final_gm_len = len(updated_data['gameMoves'])
            final_dst_len = len(updated_data['doSomethingTimestamps'])
            total_final = final_gm_len + final_dst_len

            if total_original != total_final:
                raise ValueError(
                    f"DATA MISMATCH: Original total count ({total_original}) "
                    f"does not match final total count ({total_final})."
                )

            # 4. Save the new data
            with open(output_filepath, 'w') as f:
                json.dump(updated_data, f, indent=4)

            print(f"Output to: {output_filepath}")
            print(f"Successfully updated and saved. Total moves maintained: {total_final}")

        except Exception as e:
            print(f"!!! FATAL ERROR processing {input_filepath}: {e}")

# --- Execution Cell ---

# Define the pattern to find all relevant JSON files.
# ADJUST THIS PATH if your files are in a different location.
file_pattern = '../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)/*.json'

# Get the list of all files to process
all_files_to_process = glob.glob(file_pattern)

# Execute the main function
process_files(all_files_to_process)

--- Starting file processing for 11 files ---

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_1.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_1.json
Successfully updated and saved. Total moves maintained: 472

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_10.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_10.json
Successfully updated and saved. Total moves maintained: 640

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_2.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1